#### Projected subgradient method

We extend subgradient to solve constrained convex optimization

$$\min f(x), \, \text{s.t. }x\in C$$

The `projected` subgradient step is given by

$$x^{k+1}=P(x^k-\alpha_kg^k)$$

##### Convergence analysis

Let $z^{k+1}=x^k-\alpha_kg^k$, then following analysis of subgradient method, we have

$$\begin{align*}
\|z^{k+1}-x^*\|_2^2 &\leq \|x^k-x^*\|_2^2 -2\alpha_k \left(f(x^k)-f^*\right) +\alpha_k^2\|g^k\|_2^2 \\
\end{align*}$$

Since when we project a point onto $C$, we move closer to every point in $C$, including $x^*$, therefore

$$\|x^{k+1}-x^*\|_2=\|P(z^{k+1})-x^*\|_2\leq \|z^{k+1}-x^*\|_2$$

Combine these two, we have

$$\begin{align*}
\|x^{k+1}-x^*\|_2^2 &\leq \|x^k-x^*\|_2^2 -2\alpha_k \left(f(x^k)-f^*\right) +\alpha_k^2\|g^k\|_2^2 \\
\end{align*}$$

which is the same as in subgradient method